In [15]:
import os
import gc
import glob

import numpy as np 
import pandas as pd 

from itertools import islice

from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import tensorflow as tf
#import keras.backend as K

from tensorflow.keras.layers import Dense, Lambda, Dot, Activation, Concatenate
from tensorflow.keras.layers import Layer

from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

In [11]:
NTHREADS = 4
SEED = 42
TRAIN_BATCH_SIZE = 256
TEST_BATCH_SIZE = 256
BUCKET_WINDOWS2 = [(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600)]

DATA_PATH        = '../input/optiver-realized-volatility-prediction'
BOOK_TRAIN_PATH  = '../input/optiver-realized-volatility-prediction/book_train.parquet'
TRADE_TRAIN_PATH = '../input/optiver-realized-volatility-prediction/trade_train.parquet'
BOOK_TEST_PATH   = '../input/optiver-realized-volatility-prediction/book_test.parquet'
TRADE_TEST_PATH  = '../input/optiver-realized-volatility-prediction/trade_test.parquet'
CHECKPOINT       = './model_checkpoint/model_01'

book_skip_columns = trade_skip_columns = ['time_id', 'row_id', 'target']

# 辞書を作成する

In [5]:
def get_path_dict(f, v):

    f_dict = {}
    for i in tqdm(v):
        fpath = f'{f}/stock_id={i}'
        flist = glob.glob(os.path.join(fpath, '*.parquet'))
    
        if len(flist) > 0:
            f_dict[i] = flist[0]
    
    return f_dict

In [8]:
train_ds = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
test_ds = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
train_ds['row_id'] = train_ds['stock_id'].astype(str) + '-' + train_ds['time_id'].astype(str)

print(f'Train ds shape: {train_ds.shape}')
print(f'Test ds shape: {test_ds.shape}')

Train ds shape: (428932, 4)
Test ds shape: (3, 3)


In [9]:
book_train_dict = get_path_dict(BOOK_TRAIN_PATH, train_ds['stock_id'].unique())
trade_train_dict = get_path_dict(TRADE_TRAIN_PATH, train_ds['stock_id'].unique())

book_test_dict = get_path_dict(BOOK_TEST_PATH, test_ds['stock_id'].unique())
trade_test_dict = get_path_dict(TRADE_TEST_PATH, test_ds['stock_id'].unique())

  0%|          | 0/112 [00:00<?, ?it/s]

  0%|          | 0/112 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
book_train_dict

{0: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0/c439ef22282f412ba39e9137a3fdabac.parquet',
 1: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=1/31a1c5cd6d8546b383d10373db762236.parquet',
 2: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=2/36f27d865da54e8d92ff54e07ac4afd2.parquet',
 3: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=3/ccc6d426f595409680d6f422ed911bee.parquet',
 4: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=4/6ae39a36676a419f8847cb217e5ec75d.parquet',
 5: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=5/c39da423b1404b00946ceedf32ae3f07.parquet',
 6: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=6/84352cd5e3424aeaadf11f8d89c8a07a.parquet',
 7: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=7/4f66c3336dec449280ed3719530f9486.pa

## チャンクに分ける

In [12]:
def chunks(data, SIZE=10000):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}
        
def process_book_train_chunk(chunk_ds):
    return process_optiver_ds(train_ds, chunk_ds, book_ds_fe, book_skip_columns)

def process_trade_train_chunk(chunk_ds):
    return process_optiver_ds(train_ds, chunk_ds, trade_ds_fe, trade_skip_columns)

def process_book_test_chunk(chunk_ds):
    return process_optiver_ds(test_ds, chunk_ds, book_ds_fe, book_skip_columns, False)

def process_trade_test_chunk(chunk_ds):
    return process_optiver_ds(test_ds, chunk_ds, trade_ds_fe, trade_skip_columns, False)

説明用に辞書をスキムする。

In [17]:
my_book_train_dict = { idx : book_train_dict[idx] for idx in range(5) }
my_book_train_dict

{0: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0/c439ef22282f412ba39e9137a3fdabac.parquet',
 1: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=1/31a1c5cd6d8546b383d10373db762236.parquet',
 2: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=2/36f27d865da54e8d92ff54e07ac4afd2.parquet',
 3: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=3/ccc6d426f595409680d6f422ed911bee.parquet',
 4: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=4/6ae39a36676a419f8847cb217e5ec75d.parquet'}

In [18]:
iter(my_book_train_dict)

islice は、リストの `[start:stop:step]` が iterable な要素に対して実行できる関数。

> iterable から要素を選択して返すイテレータを作成します。 start が0でない場合、iterable の要素は start に達するまでスキップされます。その後、 要素が順に返されます。

下記の例だと、0〜999の数字を要素にもつ `test_data` から、最初の10個だけを取り出す操作と同義。イテレーターを返すので、例えば for 文に入れると一つ一つの要素を見ることができる。

In [25]:
test_data = [ i for i in range(1000)]
for line in islice(test_data,10): 
    print(line)

0
1
2
3
4
5
6
7
8
9


この関数の良いところは、リストであれば `test_data[:10]` で事足りるが、iterable であれば何でも応用できるところ。

- ファイルオブジェクトに適用して、最初の10行だけを表示する
- 辞書の最初の10要素だけ取り出す（← 今回）

今回は、以下の操作に該当する。まずリスト（今回は辞書）のイテレータを作成する。外側のループは、チャンク（ひとかたまり）にしたいサイズをステップとして要素数をループするもので、チャンク数分だけ処理が加わる。次に内側のループでは、SIZE個数分だけをスライスした要素でループを回して辞書から要素を取り出している。ここでイテレータの islice を使っているので、次のチャンクで呼び出されたときにはちゃんと次のチャンクを指していることになっている。こうすることで、辞書から狙ったぶんだけスライスできる。

In [30]:
test_data = [ i for i in range(1000)]
itr = iter(test_data)
for i in range(3):
    print(">>>>>", i)
    for line in islice(itr, 5): 
        print(line)

>>>>> 0
0
1
2
3
4
>>>>> 1
5
6
7
8
9
>>>>> 2
10
11
12
13
14


In [36]:
my_book_train_chunks = [i for i in chunks(my_book_train_dict, 2)]
my_book_train_chunks

[{0: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0/c439ef22282f412ba39e9137a3fdabac.parquet',
  1: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=1/31a1c5cd6d8546b383d10373db762236.parquet'},
 {2: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=2/36f27d865da54e8d92ff54e07ac4afd2.parquet',
  3: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=3/ccc6d426f595409680d6f422ed911bee.parquet'},
 {4: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=4/6ae39a36676a419f8847cb217e5ec75d.parquet'}]

In [13]:
book_train_chunks = [i for i in chunks(book_train_dict, int(len(book_train_dict)/NTHREADS))]
trade_train_chunks = [i for i in chunks(trade_train_dict, int(len(trade_train_dict)/NTHREADS))]

z = 1 if len(book_test_dict) < NTHREADS else NTHREADS
book_test_chunks = [i for i in chunks(book_test_dict, int(len(book_test_dict)/z))]
trade_test_chunks = [i for i in chunks(trade_test_dict, int(len(trade_test_dict)/z))]

In [14]:
book_train_chunks

[{0: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0/c439ef22282f412ba39e9137a3fdabac.parquet',
  1: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=1/31a1c5cd6d8546b383d10373db762236.parquet',
  2: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=2/36f27d865da54e8d92ff54e07ac4afd2.parquet',
  3: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=3/ccc6d426f595409680d6f422ed911bee.parquet',
  4: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=4/6ae39a36676a419f8847cb217e5ec75d.parquet',
  5: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=5/c39da423b1404b00946ceedf32ae3f07.parquet',
  6: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=6/84352cd5e3424aeaadf11f8d89c8a07a.parquet',
  7: '../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=7/4f66c3336dec449280ed3719530

# np_booksを作成する

In [47]:
def calc_wap1(df):
    # Function to calculate first WAP
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap2(df):
    # Function to calculate second WAP
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_ma(df, colname, window_size):
    a = df[colname].rolling(window=window_size).mean()
    return a

def calc_mstd(df, colname, window_size):
    a = df[colname].rolling(window=window_size).std()
    return a

def calc_memw(df, colname, window_size):
    a = df[colname].ewm(span=10).mean()
    return a

def log_return(series):
    # Function to calculate the log of the return
    return np.log(series).diff()

def realized_volatility(series):
    # Calculate the realized volatility
    return np.sqrt(np.sum(series**2))

def count_unique(series):
    # Function to count unique elements of a series
    return len(np.unique(series))

In [48]:
def book_ds_fe(df):
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    #####
    win_size = 10
    bid_price1_ma = calc_ma(df, 'bid_price1', win_size)
    ask_size1_ma = calc_ma(df, 'ask_size1', win_size)
    ask_price1_ma = calc_ma(df, 'ask_price1', win_size)
    bid_size1_ma = calc_ma(df, 'bid_size1', win_size)
    
    bid_price2_ma = calc_ma(df, 'bid_price2', win_size)
    ask_size2_ma = calc_ma(df, 'ask_size2', win_size)
    ask_price2_ma = calc_ma(df, 'ask_price2', win_size)
    bid_size2_ma = calc_ma(df, 'bid_size2', win_size)
    
    df['wap1_ma'] = (bid_price1_ma * ask_size1_ma + ask_price1_ma * bid_size1_ma) / (bid_size1_ma + ask_size1_ma)
    df['wap2_ma'] = (bid_price2_ma * ask_size2_ma + ask_price2_ma * bid_size2_ma) / (bid_size2_ma + ask_size2_ma)
    
    return df

In [217]:
def np_seq_stat(a, s):
    ''' a - array, s - seconds_in_bucket'''
    
    r = []
    # BUCKET_WINDOWS2 = [(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600)]
    for w in BUCKET_WINDOWS2:
        
        idx = np.where(np.logical_and(s >= w[0], s < w[1]))[0]
       
        s_min = np.zeros((1, a.shape[1]))
        s_max = np.zeros((1, a.shape[1]))
        s_mean = np.zeros((1, a.shape[1]))
        s_std = np.zeros((1, a.shape[1]))
        s_median = np.zeros((1, a.shape[1]))
        s_sum = np.zeros((1, a.shape[1]))
        #s_mfcc = np.zeros((1, a.shape[1]))
        #s_peaks = np.zeros((1, a.shape[1]))
        
        if a[idx].shape[0] > 0:
            s_min = np.min(a[idx], axis=0, keepdims=True)
            s_max = np.max(a[idx], axis=0, keepdims=True)
            s_mean = np.mean(a[idx], axis=0, keepdims=True)
            s_std = np.std(a[idx], axis=0, keepdims=True)
            s_median = np.median(a[idx], axis=0, keepdims=True)
            s_sum = np.sum(a[idx], axis=0, keepdims=True)
            
            #s_peaks = get_number_peaks(a[idx]) # <- it gives small boost
            #s_mfcc = get_mfcc(a[idx])
            
        r.append(np.concatenate((s_min, s_max, s_mean, s_std, s_median, s_sum), axis=0))
        
    return np.nan_to_num(np.concatenate(r, axis=0).transpose())

In [115]:
def process_optiver_ds(ds, f_dict, fe_func, skip_cols, train_flg=True):
    
    x = []
    y = []
    
    for stock_id, stock_fname in tqdm(f_dict.items()):

        # book_train.csv の読み込み
        optiver_ds = pd.read_parquet(stock_fname)
        optiver_ds['row_id'] = str(stock_id) + '-' + optiver_ds['time_id'].astype(str)

        # train.csv のうち該当するストックIDだけを抽出する
        # この csv だけがターゲットの値を持っている
        sds = ds[ds['stock_id'] == stock_id]

        cols = ['time_id', 'target']
        if train_flg == False:
            cols = ['time_id']
            
        # ターゲットを持っているファイルと、offer book を左結合
        merge_ds = pd.merge(sds[cols], optiver_ds, on='time_id', how='left')
        # 結合したものに対して、fe_func (特徴量作成の関数、book_ds_fe)を適用
        merge_ds = fe_func(merge_ds).fillna(0)
        
        cols = [c for c in merge_ds.columns if c not in skip_cols]

        # numpy に変換
        # ここで time_id, stock_id, row_id は落とした量的情報だけのテーブルを np_ds にしている
        np_ds = merge_ds[cols].to_numpy(dtype=np.float16)
        
        # seconds_in_bucket, time_id をnumpyとして取り出す
        seconds_in_bucket = merge_ds['seconds_in_bucket'].to_numpy()
        
        # >>> np.unique(g_idx, return_index=True)
        # (array([    5,    11,    16, ..., 32758, 32763, 32767]),
        # array([     0,    302,    502, ..., 916830, 917018, 917325]))
        # 
        # return_index=True でユニークな値と、そのインデックス番号が返ってくる
        # いま、インデックス0とインデックス302を使用することで、time_id が 5 の情報だけをまとめることができるようになる
        g_idx = merge_ds[['time_id']].to_numpy()
        l = np.unique(g_idx, return_index=True)[1][1:]        
        
        # time_id 毎に分割
        a_list = np.split(np_ds, l) # データ (a : array)
        s_list = np.split(seconds_in_bucket, l)　# 時間情報 (s: seconds)

        stat = list(map(np_seq_stat, a_list, s_list))
        b = np.transpose(np.dstack(stat), (2, 1, 0))
        b = b.astype(np.float16)
        
        r = []
        if train_flg:
            targets = merge_ds[['target']].to_numpy(dtype=np.float16)
            t_list = np.split(targets, l)
            r = [t[0][0] for t in t_list]
        
        x.append(b)
        y.append(r)
        #break
    return x, y

In [51]:
def process_book_train_chunk(chunk_ds):
    return process_optiver_ds(train_ds, chunk_ds, book_ds_fe, book_skip_columns)

In [57]:
train_ds = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
train_ds.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


- train_ds：stock_id と time_id が書いていて、予測すべき目的変数（target）が記されている
- my_book_train_chunks：`{stock_id : file_name}` の辞書が、チャンク単位でリストになっている
- book_ds_fe：book_train から特徴量を作成する関数
- book_skip_columns：最初の方で示されている `book_skip_columns = trade_skip_columns = ['time_id', 'row_id', 'target']`、削除される要素を定義している

In [58]:
# x, y = process_optiver_ds(train_ds, my_book_train_chunks[0], book_ds_fe, book_skip_columns)

## process_optiver_ds

### book_train の読み込み

- ds : train.csv
- f_dict : チャンクに分けた辞書
- fe_func : `book_ds_fe` 関数
- skip_cols : 扱わないカラム
- train_flag : 学習か評価か
- optiver_ds : book_train.csv

実際にはループを回しているが、やっていることは

1. stock_id ごとのループを回す
1. 読み込んで、row_id (stock_id + time_id) を作成する
1. 

In [63]:
stock_id = 0
stock_fname = my_book_train_chunks[0][stock_id]

print(stock_fname)

optiver_ds = pd.read_parquet(stock_fname)
optiver_ds['row_id'] = str(stock_id) + '-' + optiver_ds['time_id'].astype(str)

optiver_ds.head()

../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0/c439ef22282f412ba39e9137a3fdabac.parquet


,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,row_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0-5
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0-5
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0-5
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0-5
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0-5


次に、train.csv (目的変数をまとめたCSVファイル）と offer book を左結合する。

1. train.csv は、いま stock_id でループを回しているので、stock_id の列はいらないので削除
1. (time_id, target) しか持たないテーブルに対して、左結合をしているので、左側表の大部分は複製されてしまうことに留意
1. 上記の注意点をおいておくと、要は target の値をもったテーブルを作りたいとうことだけ

In [214]:
ds = train_ds
print(ds.head())

sds = ds[ds['stock_id'] == stock_id]

cols = ['time_id', 'target']
merge_ds = pd.merge(sds[cols], optiver_ds, on='time_id', how='left')

print(merge_ds.shape)
merge_ds.head()

   stock_id  time_id    target
0         0        5  0.004136
1         0       11  0.001445
2         0       16  0.002168
3         0       31  0.002195
4         0       62  0.001747
(917553, 12)


,time_id,target,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,row_id
0,5,0.004136,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0-5
1,5,0.004136,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0-5
2,5,0.004136,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0-5
3,5,0.004136,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0-5
4,5,0.004136,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0-5


次に、作成した表から特徴量を作成する。

1. 特徴量（wap, log_return, volatilityなど）の計算
1. ひとつの行内の情報だけで計算できる特徴量（WAP）もあれば、ウィンドウ内での移動平均（wap_ma）を計算している特徴量もある。
1. 移動平均系は最初の部分が NaN になるので、実際にはゼロ埋めする

In [216]:
merge_ds = book_ds_fe(merge_ds)
print(merge_ds.shape)
merge_ds.head(15)

(917553, 26)


,time_id,target,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,...,wap_balance,price_spread,price_spread2,bid_spread,ask_spread,bid_ask_spread,total_volume,volume_imbalance,wap1_ma,wap2_ma
0,5,0.004136,0,1.001422,1.002301,1.001370,1.002353,3,226,2,...,0.000044,0.000878,0.000981,0.000052,-0.000052,0.000103,331,321,NaN,NaN
1,5,0.004136,1,1.001422,1.002301,1.001370,1.002353,3,100,2,...,0.000058,0.000878,0.000981,0.000052,-0.000052,0.000103,205,195,NaN,NaN
2,5,0.004136,5,1.001422,1.002301,1.001370,1.002405,3,100,2,...,0.000057,0.000878,0.001032,0.000052,-0.000103,0.000155,205,195,NaN,NaN
3,5,0.004136,6,1.001422,1.002301,1.001370,1.002405,3,126,2,...,0.000052,0.000878,0.001032,0.000052,-0.000103,0.000155,231,221,NaN,NaN
4,5,0.004136,7,1.001422,1.002301,1.001370,1.002405,3,126,2,...,0.000052,0.000878,0.001032,0.000052,-0.000103,0.000155,231,221,NaN,NaN
5,5,0.004136,11,1.001422,1.002301,1.001370,1.002405,3,100,2,...,0.000057,0.000878,0.001032,0.000052,-0.000103,0.000155,205,195,NaN,NaN
6,5,0.004136,12,1.001422,1.002301,1.001370,1.002405,3,126,2,...,0.000052,0.000878,0.001032,0.000052,-0.000103,0.000155,231,221,NaN,NaN
7,5,0.004136,14,1.001422,1.002301,1.001370,1.002405,3,126,2,...,0.000052,0.000878,0.001032,0.000052,-0.000103,0.000155,231,221,NaN,NaN
8,5,0.004136,15,1.001422,1.002301,1.001370,1.002405,3,126,2,...,0.000052,0.000878,0.001032,0.000052,-0.000103,0.000155,231,221,NaN,NaN
9,5,0.004136,16,1.001422,1.002301,1.001370,1.002405,3,126,2,...,0.000052,0.000878,0.001032,0.000052,-0.000103,0.000155,231,221,1.001442,1.001391


1. 作成したDataFrameを numpy に変換する（skip_columnsを覗いて、単純にnumpyに変換しただけ）
1. seconds_in_bucket, time_id も numpyに変換する

In [79]:
cols = [c for c in merge_ds.columns if c not in book_skip_columns]

np_ds = merge_ds[cols].to_numpy(dtype=np.float16)
seconds_in_bucket = merge_ds['seconds_in_bucket'].to_numpy()
g_idx = merge_ds[['time_id']].to_numpy()

In [110]:
# DFをnumpyにしただけのもの
# 構造は変化しない
print(np_ds.shape)
np_ds[:2]

(917553, 23)


array([[ 0.000e+00,  1.001e+00,  1.002e+00,  1.001e+00,  1.002e+00,
         3.000e+00,  2.260e+02,  2.000e+00,  1.000e+02,  1.001e+00,
         1.001e+00,        nan,        nan,  4.399e-05,  8.774e-04,
         9.804e-04,  5.174e-05, -5.174e-05,  1.035e-04,  3.310e+02,
         3.210e+02,        nan,        nan],
       [ 1.000e+00,  1.001e+00,  1.002e+00,  1.001e+00,  1.002e+00,
         3.000e+00,  1.000e+02,  2.000e+00,  1.000e+02,  1.001e+00,
         1.001e+00,  1.407e-05,  0.000e+00,  5.805e-05,  8.774e-04,
         9.804e-04,  5.174e-05, -5.174e-05,  1.035e-04,  2.050e+02,
         1.950e+02,        nan,        nan]], dtype=float16)

In [85]:
seconds_in_bucket[:5]

array([0, 1, 5, 6, 7], dtype=int16)

In [148]:
# 917553 : 全データ行数
print(g_idx.shape)
g_idx

(917553, 1)


array([[    5],
       [    5],
       [    5],
       ...,
       [32767],
       [32767],
       [32767]])

1. time_id は重複しているのでそのままでは同じ土俵でデータに組み込めない。そこでまず、g_idx として二次元配列にして、そこからユニークな値をとってきて、インデックスをとってくる
1. `[1:]` としているのは、後段で split に使用するためで、こうすることで 0〜302, 302〜502, ... といった狙い通りのスプリットができる
1. スプリットの狙いは、time_id ごとにデータを分割したいため

## np_seq_stat

- 秒数で区切って特徴量を集約する（0, 100, 200, 300, 400, 500, 600）。合計6区間を用意する
    - min, max, mean, std, median, sum の6種類を各 time window 内で計算する
    - 1 time window 6次元のベクトルができて、それを axis=0 方向に concatenate する
    - なので、stock_id毎・time_id 毎で合計36個のベクトルが縦に並ぶ（6次元×6区間）
- a_list/s_list の次元は (time_id 数 × そのid中のデータ数)

In [153]:
# time_id のユニークな値を取得
# 以下では return_index オプションを使用する
np.unique(g_idx)

array([    5,    11,    16, ..., 32758, 32763, 32767])

In [98]:
# return_index=True でユニークな値と、そのインデックス番号が返ってくる
np.unique(g_idx, return_index=True)

(array([    5,    11,    16, ..., 32758, 32763, 32767]),
 array([     0,    302,    502, ..., 916830, 917018, 917325]))

In [151]:
# [1] は index の方を取得することを意味している
np.unique(g_idx, return_index=True)[1]

array([     0,    302,    502, ..., 916830, 917018, 917325])

In [154]:
# [1:] で０を含めない
np.unique(g_idx, return_index=True)[1][1:]

array([   302,    502,    690, ..., 916830, 917018, 917325])

In [156]:
# l : time_id のユニークな値ごとのインデックス
l = np.unique(g_idx, return_index=True)[1][1:]     

# a_/s_list : time_id ごとの情報にスプリットしている
a_list = np.split(np_ds, l)
s_list = np.split(seconds_in_bucket, l)

In [157]:
# index 情報
l

array([   302,    502,    690, ..., 916830, 917018, 917325])

In [163]:
# データ
print(len(a_list))
# [0][1][2] は、各 time_id の中の情報を指す。情報量は一定ではない（もちろん特徴量の総数は一定）
print(a_list[0].shape)
print(a_list[1].shape)
print(a_list[2].shape)
a_list[0]

3830
(302, 23)
(200, 23)
(188, 23)


array([[  0.   ,   1.001,   1.002, ..., 321.   ,     nan,     nan],
       [  1.   ,   1.001,   1.002, ..., 195.   ,     nan,     nan],
       [  5.   ,   1.001,   1.002, ..., 195.   ,     nan,     nan],
       ...,
       [587.   ,   1.003,   1.004, ..., 120.   ,   1.004,   1.004],
       [588.   ,   1.003,   1.004, ...,  96.   ,   1.004,   1.004],
       [593.   ,   1.003,   1.004, ..., 120.   ,   1.004,   1.004]],
      dtype=float16)

In [165]:
# 秒数
print(a_list[0].shape, s_list[0].shape)
print(a_list[1].shape, s_list[1].shape)
print(a_list[2].shape, s_list[2].shape)

(302, 23) (302,)
(200, 23) (200,)
(188, 23) (188,)


### ウィンドウで区切る

実際の処理は、map 関数で a_list, s_list の要素に一括で行っているが、ここではその処理をいくつか分解してまとめる。

In [171]:
# map 関数 : 要素に対して一括で個別の処理を加えることができる
test_list = [1,2,3,4,5]
def test_fnc(x):
    return 2*x
list(map(test_fnc, test_list))

[2, 4, 6, 8, 10]

In [179]:
np.where(np.logical_and(0<= s_list[0], s_list[0] < 100))[0].shape
np.where(np.logical_and(0<= s_list[0], s_list[0] < 100))[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39])

各変数の初期化。np_seq_stat の中では、a_list の要素に対して処理していることに留意。以下は、ある time_id の中で、（a_list[0]なので time_id=5を指す）０〜100秒の情報だけを切り出している。その結果 40×23 の配列が確保できた。そして、該当するデータがあった場合に、それらの情報を使用してまとめた特徴量を作成する。

- time_id/time_window 内での 最大、最小、分散、平均、総和、を取る
- keepdims = True を用いることで、(1,23)の形の配列とする。
- それを縦に連結する

In [180]:
tmp = a_list[0][np.where(np.logical_and(0<= s_list[0], s_list[0] < 100))[0]]
tmp.shape

(40, 23)

In [187]:
print(np.min(tmp, axis=0, keepdims=True).shape)
tmp_min = np.min(tmp, axis=0, keepdims=True)
tmp_min

(1, 23)


array([[ 0.0000e+00,  1.0010e+00,  1.0020e+00,  1.0010e+00,  1.0020e+00,
         3.0000e+00,  1.0000e+00,  2.0000e+00,  2.0000e+00,  1.0010e+00,
         1.0010e+00,         nan,         nan,  3.3557e-05,  4.1246e-04,
         8.7690e-04,  5.1618e-05, -5.6887e-04,  1.0335e-04,  7.6000e+01,
         4.0000e+00,         nan,         nan]], dtype=float16)

In [190]:
tmp_min = np.min(tmp, axis=0, keepdims=True)
tmp_max = np.max(tmp, axis=0, keepdims=True)

np.concatenate((tmp_min, tmp_max,), axis=0).shape

(2, 23)

一つの time_id/time_window内の処理が終わると、縦に集約した情報個数分、横にもともと用意していた特徴量の種類数分の二次元配列ができる。それをリストに追加していく。

In [205]:
tmp_r = []

tmp_min = np.min(tmp, axis=0, keepdims=True)
tmp_max = np.max(tmp, axis=0, keepdims=True)

tmp_r.append(np.concatenate((tmp_min, tmp_max,), axis=0))
tmp_r.append(np.concatenate((tmp_min, tmp_max,), axis=0))

print("処理したtime_windowの個数：", len(tmp_r))
print("1 window 内の次元：（aggの種類×特徴量）", tmp_r[0].shape)
print("1 window 内の次元：（aggの種類×特徴量）", tmp_r[1].shape)

np.nan_to_num(np.concatenate(tmp_r, axis=0).transpose()).shape

処理したtime_windowの個数： 2
1 window 内の次元：（aggの種類×特徴量） (2, 23)
1 window 内の次元：（aggの種類×特徴量） (2, 23)


(23, 4)

### 最終的な出力サイズ

In [117]:
stat = list(map(np_seq_stat, a_list, s_list))

In [207]:
print(type(stat))
print(type(stat[0]))
print("stock_id=0 の中でユニークなtime_idの個数：", len(stat))
stat[0].shape

<class 'list'>
<class 'numpy.ndarray'>
stock_id=0 の中でユニークなtime_idの個数： 3830


(23, 36)

最終的に、np_seq_stat は (特徴量個数 $\times$ 集約の種類) で表される。集約は min, max, std, mean, median, sum の6個で、time_window は6つ（600秒を100秒刻み）なので、$6\times 6=36$の種類がある。

### np_seq_stat の出力を加工する

np_seq_stat は計3830個のtime_id の情報を持ったリストで、一つひとつの要素は $36\times 23$ の次元を持っている。これを連結して numpy 配列にする

In [208]:
b = np.transpose(np.dstack(stat), (2, 1, 0))
b = b.astype(np.float16)

print(b.shape)

(3830, 36, 23)


## 正解データの作成

- merge_ds から target の値をとってくる
- このままだと、全データ行分の次元を持っているので、実際の個数に合わせる必要がある
- 以上までで time_id で split したことを思い出して、同様に split する
- split しただけでは、さらに seconds_in_bucket 分の次元を持っているので、一つを代表してループを回す
- こうすることで、stock_id に対しての正解ができた

In [213]:
tmp_r = []

targets = merge_ds[['target']].to_numpy(dtype=np.float16)
print(targets.shape)

t_list = np.split(targets, l)
print(len(t_list))
print(t_list[0].shape)

tmp_r = [t[0][0] for t in t_list]

print(len(tmp_r))

(917553, 1)
3830
(302, 1)
3830


# サイズ

In [150]:
# 正解データ
print(train_ds.shape)
print("\n")

# 元データ --> stock_id 毎にこのDataFrameは作成している
# time_id, stock_id, row_id 分、np_ds よりも多い
print(type(merge_ds))
print(merge_ds.shape)
print("\n")

# データ
print(type(np_ds))
print(np_ds.shape)
print("\n")

# np_ds ---> a_list で、time_id ごとに切っている
# time_id 毎に、np_ds を分割したもの
print(type(a_list))
print(type(a_list[0]))
print(len(a_list))
print(a_list[0].shape)
print(a_list[1].shape)

(428932, 3)


<class 'pandas.core.frame.DataFrame'>
(917553, 26)


<class 'numpy.ndarray'>
(917553, 23)


<class 'list'>
<class 'numpy.ndarray'>
3830
(302, 23)
(200, 23)


# process_optiver_ds

- stock_id でループ
- train/book のCSVをマージする
- 不要なカラムを削除
- time_id 毎に分けるために、 `np.split(np_ds, l)` を実行
- b (data size × time-step × feature dim)
- 最終出力：(stock_id × data_size) × time-step × feature-dim

# LSTM

- 想定する入力データshape: (batch_size, timesteps, data_dim)
- input_shape=(timesteps, data_dim)